<a href="https://colab.research.google.com/github/sunq99/ML_GroupPJ/blob/main/3%EC%A1%B0_%EB%AA%A8%EB%8D%B8_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn==1.3.0
!pip install optuna
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

svr 모델링

In [ ]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 데이터 불러오기
data = pd.read_csv('/content/fffffffffff (1).csv', encoding='cp949')
data.set_index('법정동코드', inplace=True)

# 범주형 데이터 라벨 인코딩
categorical_cols = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# X, y 분할
X = data.drop(columns=['y'])  # <--- 들여쓰기 수정됨
y = data[['y']].astype(float)

# 🔹 데이터 정규화 (SVR에 필요)
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# K-Fold 설정
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# ===============================
# 🔥 SVR 하이퍼파라미터 튜닝 (Optuna, RMSE 최소화)
# ===============================
def objective_svr(trial):
    params = {
        'C': trial.suggest_float('C', 0.01, 50.0, log=True),  # 더 넓은 범위
        'epsilon': trial.suggest_float('epsilon', 0.5, 1.0),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid']),
    }

    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 7)  # 다항식 차수 추가

    model = SVR(**params)

    # K-Fold 교차 검증 수행
    rmse_scores = []
    for train_idx, val_idx in kfold.split(X_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr.values.ravel())  # 모델 학습
        y_val_pred = model.predict(X_val)  # 검증 데이터 예측

        # RMSE 계산 (MSE의 제곱근)
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)  # 평균 RMSE 반환 (최소화 목표)

# Optuna 실행 (RMSE 최소화)
study_svr = optuna.create_study(direction="minimize")  # ⬅️ 방향 변경
try:
    study_svr.optimize(objective_svr, n_trials=350, show_progress_bar=True, gc_after_trial=True, catch=(Exception,))
except Exception as e:
    print("⚠ Optuna 실행 중 오류 발생:", e)

# 최적의 하이퍼파라미터 확인
if study_svr.best_trial:
    best_svr_params = study_svr.best_params
    print("Best SVR Parameters (최소 RMSE 기준):", best_svr_params)

    # 최적의 SVR 모델 생성
    svr_best = SVR(**best_svr_params)

    # 모델 학습
    svr_best.fit(X_train, y_train.values.ravel())

    # 학습 데이터에 대한 예측
    y_train_pred = svr_best.predict(X_train)

    # 학습 데이터 RMSE 계산
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

    # 학습 및 테스트 R² 점수
    train_score = svr_best.score(X_train, y_train)
    test_score = svr_best.score(X_test, y_test)

    print("\n🔥 SVR 최종 성능 (최소 RMSE 기준):")
    print(f"  - 학습 데이터 R²: {train_score:.4f}")
    print(f"  - 테스트 데이터 R²: {test_score:.4f}")
    print(f"  - 학습 데이터 RMSE: {rmse_train:.4f}")  # 학습 데이터 RMSE 출력

    # 예측 및 평가 지표 계산
    y_test_pred = svr_best.predict(X_test)

    mae_test = mean_absolute_error(y_test, y_test_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    rmse_test = np.sqrt(mse_test)  # RMSE 추가
    r2_test = r2_score(y_test, y_test_pred)

    print("\n📊 평가 지표:")
    print(f"  - 테스트 MAE: {mae_test:.4f}")
    print(f"  - 테스트 MSE: {mse_test:.4f}")
    print(f"  - 테스트 RMSE: {rmse_test:.4f}")  # 테스트 RMSE 출력
    print(f"  - 테스트 R²: {r2_test:.4f}")
else:
    print("⚠ 최적의 하이퍼파라미터를 찾지 못했습니다.")


HTP 이후 모델링

In [ ]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 데이터 불러오기
data = pd.read_csv('/content/final.csv', encoding='cp949')
data.set_index('법정동코드', inplace=True)

# 범주형 데이터 라벨 인코딩
categorical_cols = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# X, y 분할
X = data.drop(columns=['y'])
y = data[['y']].astype(float)

# 🔹 데이터 정규화 (SVR에 필요)
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# K-Fold 설정
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# ===============================
# 🔥 SVR 하이퍼파라미터 튜닝 (Optuna, RMSE 최소화)
# ===============================
def objective_svr(trial):
    params = {
        'C': trial.suggest_float('C', 0.01, 50.0, log=True),  # 더 넓은 범위
        'epsilon': trial.suggest_float('epsilon', 0.5, 1.0),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid']),
    }

    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 7)  # 다항식 차수 추가

    model = SVR(**params)

    # K-Fold 교차 검증 수행
    rmse_scores = []
    for train_idx, val_idx in kfold.split(X_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model.fit(X_tr, y_tr.values.ravel())  # 모델 학습
        y_val_pred = model.predict(X_val)  # 검증 데이터 예측

        # RMSE 계산 (MSE의 제곱근)
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)  # 평균 RMSE 반환 (최소화 목표)

# Optuna 실행 (RMSE 최소화)
study_svr = optuna.create_study(direction="minimize")  # ⬅️ 방향 변경
try:
    study_svr.optimize(objective_svr, n_trials=350, show_progress_bar=True, gc_after_trial=True, catch=(Exception,))
except Exception as e:
    print("⚠ Optuna 실행 중 오류 발생:", e)

# 최적의 하이퍼파라미터 확인
if study_svr.best_trial:
    best_svr_params = study_svr.best_params
    print("Best SVR Parameters (최소 RMSE 기준):", best_svr_params)

    # 최적의 SVR 모델 생성
    svr_best = SVR(**best_svr_params)

    # 모델 학습
    svr_best.fit(X_train, y_train.values.ravel())

    # 학습 데이터에 대한 예측
    y_train_pred = svr_best.predict(X_train)

    # 학습 데이터 RMSE 계산
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

    # 학습 및 테스트 R² 점수
    train_score = svr_best.score(X_train, y_train)
    test_score = svr_best.score(X_test, y_test)

    print("\n🔥 SVR 최종 성능 (최소 RMSE 기준):")
    print(f"  - 학습 데이터 R²: {train_score:.4f}")
    print(f"  - 테스트 데이터 R²: {test_score:.4f}")
    print(f"  - 학습 데이터 RMSE: {rmse_train:.4f}")  # 학습 데이터 RMSE 출력

    # 예측 및 평가 지표 계산
    y_test_pred = svr_best.predict(X_test)

    mae_test = mean_absolute_error(y_test, y_test_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    rmse_test = np.sqrt(mse_test)  # RMSE 추가
    r2_test = r2_score(y_test, y_test_pred)

    print("\n📊 평가 지표:")
    print(f"  - 테스트 MAE: {mae_test:.4f}")
    print(f"  - 테스트 MSE: {mse_test:.4f}")
    print(f"  - 테스트 RMSE: {rmse_test:.4f}")  # 테스트 RMSE 출력
    print(f"  - 테스트 R²: {r2_test:.4f}")
else:
    print("⚠ 최적의 하이퍼파라미터를 찾지 못했습니다.")


그래프로 파라미터 확인

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history

# 기존 코드 실행
fig = plot_optimization_history(study_svr)
fig.update_xaxes(range=[0, 360])
fig.update_yaxes(range=[1.4495, 1.4497])
fig.add_vline(
    x=study_svr.best_trial.number,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Trial {study_svr.best_trial.number}",
    annotation_position="top"
)
fig.show()

모델 성능 평가

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

data = pd.read_csv('/content/fffffffffff (1).csv', encoding='cp949')

if '법정동코드' in data.columns:
    data.set_index('법정동코드', inplace=True)

categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

X = data.drop(columns=['y'], errors='ignore')
y = data[['y']].astype(float)

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

best_svr_params = {
    'C': 10.57463847100441,
    'epsilon': 0.001,
    'kernel': 'rbf',
}

# ✅ SVR 모델 생성 및 학습
svr_best = SVR(**best_svr_params)
svr_best.fit(X_train, y_train.values.ravel())

#=================데이터 삽입===============
test_m = pd.read_csv('/content/model_test.csv', encoding='cp949')
#================여기를 수정================
test_X = test_m.drop(columns=['법정동코드', 'y'], errors='ignore')

le = LabelEncoder()
test_X['산후조리업수_구간'] = le.fit_transform(test_X['산후조리업수_구간'])

test_X_scaled = pd.DataFrame(scaler.transform(test_X), columns=test_X.columns)

y_pred = svr_best.predict(test_X_scaled)

print("📢 예측 결과:", y_pred)


SHAP

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False
explainer = shap.KernelExplainer(svr_best.predict, X_train)

    # SHAP 값 계산 (예: 100개의 샘플에 대해 설명)
shap_values = explainer.shap_values(X_test[:100])
# SHAP summary plot
shap.summary_plot(shap_values, X_test[:100])

# 학습 데이터에 대한 예측
y_train_pred = svr_best.predict(X_train)

# 학습 데이터 RMSE 계산
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

# 학습 및 테스트 R² 점수
train_score = svr_best.score(X_train, y_train)
test_score = svr_best.score(X_test, y_test)

print("\n🔥 SVR 최종 성능 (최소 RMSE 기준):")
print(f"  - 학습 데이터 R²: {train_score:.4f}")
print(f"  - 테스트 데이터 R²: {test_score:.4f}")
print(f"  - 학습 데이터 RMSE: {rmse_train:.4f}")  # 학습 데이터 RMSE 출력

# 예측 및 평가 지표 계산
y_test_pred = svr_best.predict(X_test)

mae_test = mean_absolute_error(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)  # RMSE 추가
r2_test = r2_score(y_test, y_test_pred)

print("\n📊 평가 지표:")
print(f"  - 테스트 MAE: {mae_test:.4f}")
print(f"  - 테스트 MSE: {mse_test:.4f}")
print(f"  - 테스트 RMSE: {rmse_test:.4f}")  # 테스트 RMSE 출력
print(f"  - 테스트 R²: {r2_test:.4f}")
